# PDE-Selector: Paper Reproduction Notebook

This notebook reproduces all results from the paper:

> **A Meta-Learning Framework for Automated Selection of PDE Identification Methods**

## What This Notebook Does

1. **Loads frozen dataset** and verifies statistics
2. **Trains the Random Forest selector** and compares against baselines
3. **Computes regret** (both in-sample and test set, clearly labeled)
4. **Regenerates all paper figures** via official scripts
5. **Runs noise stress test** (optional, ~10 min)
6. **Verifies paper consistency** via automated checks

**Key Results:**
- Test accuracy: 97.06%
- In-sample (full dataset) zero-regret: 99.41%
- Test set zero-regret: 97.06%

---
## Setup: Robust Repository Root Detection

In [ ]:
import subprocess
import sys
from pathlib import Path

def find_repo_root(start_path=None, markers=('Makefile', 'pyproject.toml', '.git')):
    """
    Walk up from start_path until we find a repo marker file.
    Works regardless of where Jupyter was launched from.
    """
    if start_path is None:
        start_path = Path.cwd()
    
    current = Path(start_path).resolve()
    
    # Walk up to filesystem root
    while current != current.parent:
        for marker in markers:
            marker_path = current / marker
            if marker_path.exists():
                return current
        current = current.parent
    
    # Check root directory too
    for marker in markers:
        if (current / marker).exists():
            return current
    
    raise RuntimeError(
        f"Could not find repo root. Run this notebook from inside the repository.\n"
        f"Looked for markers: {markers}\n"
        f"Started from: {start_path}"
    )

REPO_ROOT = find_repo_root()
print(f"✓ Repository root: {REPO_ROOT}")

# Verify key directories exist
assert (REPO_ROOT / 'scripts').exists(), f"scripts/ not found at {REPO_ROOT}"
assert (REPO_ROOT / 'data').exists(), f"data/ not found at {REPO_ROOT}"
print(f"✓ Verified: scripts/, data/, manuscript/ directories exist")

# Add src to path for imports
sys.path.insert(0, str(REPO_ROOT))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

# Paths
DATA_DIR = REPO_ROOT / 'data'
RESULTS_DIR = DATA_DIR / 'results'
FIGURES_DIR = REPO_ROOT / 'manuscript' / 'figures'
SCRIPTS_DIR = REPO_ROOT / 'scripts'
PAPER_RUN = REPO_ROOT / 'experiments' / 'paper_run_2025-12-18'

print(f"✓ Data directory: {DATA_DIR}")
print(f"✓ Figures directory: {FIGURES_DIR}")

---
## Step 1: Load Frozen Dataset

In [ ]:
# Load the frozen dataset from paper run
dataset_path = PAPER_RUN / 'full_dataset_4methods.csv'
if not dataset_path.exists():
    dataset_path = RESULTS_DIR / 'full_dataset_4methods.csv'

df = pd.read_csv(dataset_path)
print(f"Dataset: {dataset_path.name}")
print(f"Shape: {df.shape[0]:,} windows × {df.shape[1]} columns")
print(f"\nPDE Distribution:")
print(df['pde_type'].value_counts().to_string())

In [ ]:
# Best method distribution (paper Table 1)
print("\n=== Best Method Distribution ===")
best_counts = df['best_method'].value_counts()
best_pct = (best_counts / len(df) * 100).round(2)
for method in best_counts.index:
    print(f"  {method}: {best_counts[method]:,} ({best_pct[method]:.1f}%)")
print(f"\nTotal: {len(df):,} windows")

---
## Step 2: Train Selector (Random Forest)

Exactly replicates `scripts/train_models.py` with proper train/test split.

**Critical**: We split indices ONCE and reuse them to ensure alignment between X, y, and df rows.

In [ ]:
# Extract features and labels
feature_cols = [f'feat_{i}' for i in range(12)]
X = df[feature_cols].values
y = df['best_method'].values

# CRITICAL: Split indices ONCE and reuse for perfect alignment
indices = np.arange(len(df))
idx_train, idx_test = train_test_split(
    indices, test_size=0.2, random_state=42, stratify=y
)

# Build aligned X, y, and df subsets
X_train = X[idx_train]
X_test = X[idx_test]
y_train = y[idx_train]
y_test = y[idx_test]
df_train = df.iloc[idx_train].reset_index(drop=True)
df_test = df.iloc[idx_test].reset_index(drop=True)

# Verify alignment
assert len(X_train) == len(df_train) == len(y_train)
assert len(X_test) == len(df_test) == len(y_test)
assert np.array_equal(df_test['best_method'].values, y_test), "y_test misaligned with df_test!"

# Standardize features (fit on train only)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {len(X_train):,} samples")
print(f"Test set: {len(X_test):,} samples")
print(f"\n✓ Alignment verified: X, y, and df subsets match")
print(f"\nTest set class distribution:")
unique, counts = np.unique(y_test, return_counts=True)
for cls, cnt in zip(unique, counts):
    print(f"  {cls}: {cnt}")

In [ ]:
# Train and compare models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'KNN (k=5)': KNeighborsClassifier(n_neighbors=5),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'SVM (RBF)': SVC(kernel='rbf', random_state=42),
    'Ridge Classifier': RidgeClassifier(random_state=42),
}

results = []
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    test_acc = accuracy_score(y_test, y_pred)
    results.append({'Model': name, 'Test Accuracy': test_acc})
    print(f"{name}: {test_acc:.4f}")

results_df = pd.DataFrame(results).sort_values('Test Accuracy', ascending=False)
print(f"\n✓ Best model: {results_df.iloc[0]['Model']} ({results_df.iloc[0]['Test Accuracy']:.2%})")

---
## Step 3: Compute Regret (In-Sample and Test Set)

**Regret** = selector's error − oracle's error. Zero regret means the selector achieves oracle-level error.

In [ ]:
# Train best model (Random Forest)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

In [ ]:
# === IN-SAMPLE REGRET (full dataset) ===
X_full_scaled = scaler.transform(X)
predictions_full = rf_model.predict(X_full_scaled)

regrets_full = []
for idx in range(len(df)):
    pred_method = predictions_full[idx]
    selector_e2 = df.iloc[idx][f'{pred_method}_e2']
    oracle_e2 = df.iloc[idx]['oracle_e2']
    regrets_full.append(selector_e2 - oracle_e2)
regrets_full = np.array(regrets_full)

print("="*50)
print("IN-SAMPLE REGRET (Full Dataset)")
print("="*50)
print(f"Zero-regret count: {np.sum(regrets_full == 0):,} / {len(regrets_full):,}")
print(f"Zero-regret rate: {100*np.mean(regrets_full == 0):.2f}%")
print(f"Mean regret: {regrets_full.mean():.6f}")
print(f"Max regret: {regrets_full.max():.4f}")

In [ ]:
# === TEST SET REGRET (held-out, properly aligned) ===
predictions_test = rf_model.predict(X_test_scaled)

# Verify alignment before computing regret
assert len(predictions_test) == len(df_test), f"Length mismatch: {len(predictions_test)} vs {len(df_test)}"

regrets_test = []
for idx in range(len(df_test)):
    pred_method = predictions_test[idx]
    selector_e2 = df_test.iloc[idx][f'{pred_method}_e2']
    oracle_e2 = df_test.iloc[idx]['oracle_e2']
    regrets_test.append(selector_e2 - oracle_e2)
regrets_test = np.array(regrets_test)

print("="*50)
print("HELD-OUT TEST SET REGRET")
print("="*50)
print(f"Zero-regret count: {np.sum(regrets_test == 0):,} / {len(regrets_test):,}")
print(f"Zero-regret rate: {100*np.mean(regrets_test == 0):.2f}%")
print(f"Test accuracy: {accuracy_score(y_test, predictions_test):.2%}")
print(f"Mean regret: {regrets_test.mean():.6f}")

In [ ]:
# Sanity check: show first 5 test predictions
print("\n=== Sanity Check: First 5 Test Predictions ===")
print(f"{'Row':<5} {'True Label':<12} {'Predicted':<12} {'Oracle E2':<12} {'Selected E2':<12} {'Regret':<10}")
print("-" * 65)
for i in range(min(5, len(df_test))):
    true_label = y_test[i]
    pred_label = predictions_test[i]
    oracle_e2 = df_test.iloc[i]['oracle_e2']
    selected_e2 = df_test.iloc[i][f'{pred_label}_e2']
    regret = selected_e2 - oracle_e2
    print(f"{i:<5} {true_label:<12} {pred_label:<12} {oracle_e2:<12.6f} {selected_e2:<12.6f} {regret:<10.6f}")

---
## Step 4: Regenerate Paper Figures

Runs the official `scripts/generate_figures.py` to produce all paper figures.

In [ ]:
# Generate all figures via official script
print("Generating figures via scripts/generate_figures.py...")
result = subprocess.run(
    [sys.executable, 'generate_figures.py'],
    cwd=SCRIPTS_DIR,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✓ Figures generated successfully")
else:
    print(f"✗ Error: {result.stderr}")

In [ ]:
# Verify all required figures exist
required_figures = [
    'confusion_matrix.png',
    'feature_importance.png',
    'method_distribution.png',
    'model_comparison.png',
    'regret_cdf.png',
    'noise_best_method_distribution.png'
]

print("Checking manuscript/figures/:")
all_exist = True
for fig in required_figures:
    path = FIGURES_DIR / fig
    if path.exists():
        print(f"  ✓ {fig}")
    else:
        print(f"  ✗ {fig} MISSING")
        all_exist = False

if all_exist:
    print("\n✓ All paper figures present")

In [ ]:
# Display key figures
print("=== Confusion Matrix (Test Set) ===")
display(Image(filename=str(FIGURES_DIR / 'confusion_matrix.png'), width=500))

In [ ]:
print("=== Regret CDF (In-Sample) ===")
display(Image(filename=str(FIGURES_DIR / 'regret_cdf.png'), width=500))

In [ ]:
print("=== Feature Importance ===")
display(Image(filename=str(FIGURES_DIR / 'feature_importance.png'), width=500))

---
## Step 5: Noise Stress Test (Optional)

Reproduces the noise/outlier stress test from Section 4.2 of the paper.

⚠️ Takes ~10 minutes. Set `RUN_STRESS_TEST = True` to execute.

In [ ]:
RUN_STRESS_TEST = False  # Set to True for full reproduction

if RUN_STRESS_TEST:
    print("Running noise stress test (~10 min)...")
    result = subprocess.run(
        [sys.executable, 'noise_stress_test.py'],
        cwd=SCRIPTS_DIR,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print("✓ Stress test complete")
        print(result.stdout[-500:] if len(result.stdout) > 500 else result.stdout)
    else:
        print(f"✗ Error: {result.stderr}")
else:
    print("Skipping stress test. Set RUN_STRESS_TEST = True to run.")
    print("Loading existing results...")

In [ ]:
# Display stress test results
stress_csv = RESULTS_DIR / 'noise_stress_test_summary.csv'
if stress_csv.exists():
    stress_df = pd.read_csv(stress_csv)
    print("=== Noise Stress Test Summary ===")
    print(stress_df.to_string(index=False))
else:
    print(f"Stress test results not found at {stress_csv}")

In [ ]:
# Display noise figure
noise_fig = FIGURES_DIR / 'noise_best_method_distribution.png'
if noise_fig.exists():
    print("=== Noise Best Method Distribution ===")
    display(Image(filename=str(noise_fig), width=600))
else:
    print("Noise figure not found. Run stress test to generate.")

---
## Step 6: Automated Consistency Check

Verifies all paper-referenced files exist and metrics match.

In [ ]:
print("Running paper consistency check...")
result = subprocess.run(
    [sys.executable, 'check_paper_consistency.py'],
    cwd=SCRIPTS_DIR,
    capture_output=True,
    text=True
)

print(result.stdout)
if result.returncode == 0:
    print("\n" + "="*50)
    print("✓ ALL PAPER CONSISTENCY CHECKS PASSED")
    print("="*50)
else:
    print(f"\n✗ CHECKS FAILED")
    print(result.stderr)

---
## Summary

This notebook reproduced all key results from the paper.

In [ ]:
# Final summary table
summary = {
    'Metric': [
        'Dataset size',
        'PDEs',
        'Methods compared',
        'Best classifier',
        'Test accuracy',
        'In-sample zero-regret',
        'Test set zero-regret',
        'Mean regret (in-sample)'
    ],
    'Value': [
        f"{len(df):,} windows",
        'KdV, Heat, KS, Transport',
        'LASSO, STLSQ, WeakIDENT, RobustIDENT',
        'Random Forest',
        f"{accuracy_score(y_test, predictions_test):.2%}",
        f"{100*np.mean(regrets_full == 0):.2f}%",
        f"{100*np.mean(regrets_test == 0):.2f}%",
        f"{regrets_full.mean():.6f}"
    ]
}

summary_df = pd.DataFrame(summary)
print("\n" + "="*50)
print("REPRODUCTION SUMMARY")
print("="*50)
print(summary_df.to_string(index=False))
print("\n✓ Reproduction complete!")